# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Oct1222_postcalibration")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL4-FI2 (1)_CRR_DiadFit,103.002584,0.017829,0.012922,1286.133629,0.010920,121.225132,1286.133629,110.041153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL4-FI2_CRR_DiadFit,103.001113,0.020921,0.015469,1286.142803,0.013659,116.596615,1286.142803,118.529969,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL4-FI3-4-#1_CRR_DiadFit,103.086210,0.042680,0.030803,1285.908974,0.025700,61.341563,1285.908974,56.048735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL4-FI3-4-#1rep,103.095987,0.004845,0.004845,1285.905372,0.000000,327.309480,1285.905372,361.326064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL4-FI3-4-#2,103.088796,0.014191,0.010220,1285.874144,0.008467,214.507375,1285.874144,252.089897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL4-FI2 (1).txt',
 'LL4-FI2.txt',
 'LL4-FI3-4-#1.txt',
 'LL4-FI3-4-#1rep.txt',
 'LL4-FI3-4-#2.txt',
 'LL7-107-#2-rep1.txt',
 'LL7-107-#2-rep2.txt',
 'LL7-107-#2-rep3.txt',
 'LL7-107-#2-rep4.txt',
 'LL7-107-#2-rep5.txt',
 'LL7-19dp-#1.txt',
 'LL7-19dp-#2.txt',
 'LL7-19dp-#3.txt',
 'LL7-19dp-#4.txt',
 'LL7-19dp-#5.txt',
 'LL7-19dp-#6.txt',
 'LL7-19dp-#7.txt',
 'LL7-20.txt',
 'LL8-FI38-#1.txt',
 'de (1).txt',
 'de (2).txt',
 'de (3).txt',
 'de (4).txt',
 'de.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 24/24 [00:05<00:00,  4.67it/s]

Done


'LL7-107-#2-rep1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL7-107-#2-rep1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL4-FI2 (1)', 'LL4-FI2', 'LL4-FI3-4-#1', 'LL4-FI3-4-#1rep',
       'LL4-FI3-4-#2', 'LL7-19dp-#1', 'LL7-19dp-#2', 'LL7-19dp-#3',
       'LL7-19dp-#4', 'LL7-19dp-#5', 'LL7-19dp-#6', 'LL7-19dp-#7',
       'LL7-20', 'LL7-107-#2-rep1', 'LL7-107-#2-rep2', 'LL7-107-#2-rep3',
       'LL7-107-#2-rep4', 'LL7-107-#2-rep5', 'LL8-FI38-#1'], dtype='<U15')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,57489.0,0.997802,0.997797,0.997807
1,57246.0,0.997803,0.997798,0.997809
2,58779.0,0.997793,0.997788,0.997798
3,59109.0,0.997791,0.997785,0.997796
4,59661.0,0.997787,0.997782,0.997792


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y
0,LL4-FI2 (1).txt,1151.300532,850.244031,406.407031,Spline,NaN,NaN,NaN,NaN
1,LL4-FI2.txt,1151.300532,864.893011,414.774952,Spline,NaN,NaN,NaN,NaN
2,LL4-FI3-4-#1rep.txt,1151.064553,153.598242,103.367615,Spline,NaN,NaN,NaN,NaN
3,LL4-FI3-4-#2.txt,1151.035056,180.775038,98.393266,Spline,NaN,NaN,NaN,NaN
4,LL7-19dp-#5.txt,NaN,NaN,NaN,NaN,1086.983353,378.611848,9.618044,PseudoVoigtModel


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL4-FI2 (1)', 'LL4-FI2', 'LL4-FI3-4-#1rep', 'LL4-FI3-4-#2',
       'LL7-19dp-#5', 'LL7-19dp-#6'], dtype=object)

In [16]:
df_combo['name_for_matching']

0         LL4-FI2 (1)
1             LL4-FI2
2        LL4-FI3-4-#1
3     LL4-FI3-4-#1rep
4        LL4-FI3-4-#2
5         LL7-19dp-#1
6         LL7-19dp-#2
7         LL7-19dp-#3
8         LL7-19dp-#4
9         LL7-19dp-#5
10        LL7-19dp-#6
11        LL7-19dp-#7
12             LL7-20
13    LL7-107-#2-rep1
14    LL7-107-#2-rep2
15    LL7-107-#2-rep3
16    LL7-107-#2-rep4
17    LL7-107-#2-rep5
18        LL8-FI38-#1
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name_x,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Model_name_y,Carb_Diad_Ratio,SO2_Diad_Ratio
0,0,102.776157,102.789061,102.763253,0.012904,LL4-FI2 (1)_CRR_DiadFit,103.002584,0.017829,0.012922,1286.133629,...,1151.300532,850.244031,406.407031,Spline,NaN,NaN,NaN,NaN,NaN,3.047156
1,0,102.774867,102.790311,102.759423,0.015444,LL4-FI2_CRR_DiadFit,103.001113,0.020921,0.015469,1286.142803,...,1151.300532,864.893011,414.774952,Spline,NaN,NaN,NaN,NaN,NaN,2.971719
2,0,102.858684,102.889423,102.827945,0.030739,LL4-FI3-4-#1_CRR_DiadFit,103.086210,0.042680,0.030803,1285.908974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,102.868212,102.873075,102.863349,0.004863,LL4-FI3-4-#1rep,103.095987,0.004845,0.004845,1285.905372,...,1151.064553,153.598242,103.367615,Spline,NaN,NaN,NaN,NaN,NaN,0.166848
4,0,102.860666,102.870878,102.850454,0.010212,LL4-FI3-4-#2,103.088796,0.014191,0.010220,1285.874144,...,1151.035056,180.775038,98.393266,Spline,NaN,NaN,NaN,NaN,NaN,0.283798


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()